# 올리브영 상세 정보 및 리뷰 크롤링

In [ ]:
import pandas as pd
# # 모든 열 너비를 표시
# pd.set_option('display.max_colwidth', None)
# # 옵션을 기본값으로 재설정
# pd.reset_option('display.max_colwidth')

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

CHROME_DRIVER_PATH = "C:\\Users\\82108\\.wdm\\drivers\\chromedriver\\win64\\127.0.6533.72\\chromedriver-win32\\chromedriver.exe"
service = Service(executable_path=CHROME_DRIVER_PATH)
options = Options()
driver = webdriver.Chrome(service=service, options=options)

name_list = []
brand_list = []
price_list = []
sale_price_list = []
picture_list = []
url_list = []
volume_list = []
skin_type_list = []
ingredient_list = []
review_list = []
all_review_list = []

# 각 제품을 빈 리스트에 추가
def make_list(name, brand, price, sale_price, picture, url, volume, skin_type, ingredient, indiv_product_reviews):
    name_list.append(name)
    brand_list.append(brand)
    price_list.append(price)
    sale_price_list.append(sale_price)
    picture_list.append(picture)
    url_list.append(url)
    volume_list.append(volume)
    skin_type_list.append(skin_type)
    ingredient_list.append(ingredient)
    review_list.append(indiv_product_reviews)

# 리뷰 페이지 내에서 다음 페이지 클릭
def review_next_page(review_page_num):
    try:
        page_bar = driver.find_elements(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing a')
        for button in page_bar:
            if button.get_attribute("data-page-no") == str(review_page_num):
                driver.execute_script("arguments[0].click();", button)
                # WebDriverWait(driver, 10).until(
                #     EC.presence_of_element_located((By.CSS_SELECTOR, '#gdasList')))

                break
    except NoSuchElementException:
        pass


def extract_review(driver,name):
    # 최신순으로 정렬하는 코드
    try: 
        sort_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="gdasSort"]/li[3]/a')))
        driver.execute_script("arguments[0].click();", sort_button)
        WebDriverWait(driver, 5)
    
        # 체험단 제거하는 코드
        ad_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="searchType_3"]')))
        ad_button.click()    
    
        # 모든 페이지 버튼 요소를 찾기
        page_buttons = driver.find_elements(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing a')
        # 각 제품의 리뷰 하나의 리스트로 담기
        indiv_product_reviews = []
        # 리뷰 페이지에 한 페이지 당 10개의 리뷰 존재
        for review_page_num in range(1, 11): 
            try:
                review_next_page(review_page_num)
                WebDriverWait(driver, 60).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#gdasList > li > div.review_cont > div.txt_inner')))
                for review_num in range(1, 11):
                    # 몇개의 리뷰에서 (리뷰 + 타이틀, 리뷰, 타이틀) >> 3개의 형식으로 리뷰가 작성됨
                    # 리뷰와 타이틀을 더해줌
                    # 특정 날짜 이후로 타이틀이 없어지고, 그 타이틀 패스가 바로 및의 문장으로 변경 됨
                    #  ※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후~~
                    # 전처리할 때 없애줘야 할거 같음
                    try:
                        # 리뷰 타이틀과 내용을 초기화
                        review_title = ""
                        review_content = ""

                        # 리뷰 타이틀 추출
                        try:
                            review_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({review_num}) > div.review_cont > p').text
                        except (NoSuchElementException, StaleElementReferenceException):
                            pass

                        # 리뷰 내용 추출
                        try:
                            review_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({review_num}) > div.review_cont > div.txt_inner').text
                        except (NoSuchElementException, StaleElementReferenceException):
                            pass

                        # 타이틀과 내용을 합쳐서 리뷰 리스트에 추가
                        if review_title or review_content:
                            review = [f"{review_title} {review_content}".strip()]
                            indiv_product_reviews.append(review)
                            all_review_list.append(review)

                    except (NoSuchElementException, StaleElementReferenceException):
                        pass
            except (NoSuchElementException, StaleElementReferenceException):
                break
              
        return indiv_product_reviews
    except:
        print(name)


# 제품 내 모든 정보 추출
# para >> 제품 목록에 들어가면 문단(행?)으로 path가 달라짐
# 첫 페이지에서 48개의 제품이 보이게 했을 때 문단 8~19의 숫자로 path가 달라짐
# product_number >> 열 방향?으로도 달라짐.가로 방향으로 1~4씩 바뀜
def extract_data(driver, para, product_number):
    try:
        # 제품 클릭
        product_button = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({para}) > li:nth-child({product_number}) > div > div > a > p') 
        product_button.click()
        
        # 제품 명 추출
        name = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name'))).text
        
        # url 추출
        url = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'head > link')))
        url = url.get_attribute('href')
        
        # 이미지 추출
        # 2번째 이미지 먼저 추출. 없을 시 첫 번째 이미지 추출
        try:
            # img 요소를 찾기(두 번째 사진)
            img_element = driver.find_element(By.CSS_SELECTOR, '#prd_thumb_list > li:nth-child(3) > a')
            # src 속성 값 가져오기
            picture = img_element.get_attribute('data-img')
        except:
            # img 요소를 찾기
            img_element = driver.find_element(By.ID, 'mainImg')
            # src 속성 값 가져오기
            picture = img_element.get_attribute('src')

        # 브랜드 명 추출
        brand = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#moveBrandShop'))).text

        # 가격 추출
        # 할인된 가격이 없을 시, None으로 대체
        try:
            sale_price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong')  
            sale_price = sale_price_element.text
            price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike') 
            price = price_element.text
        except NoSuchElementException:
            sale_price = None
            price_element = driver.find_element(By.CSS_SELECTOR, '#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong') 
            price = price_element.text


        # 구매정보 클릭(상세 정보보)
        info_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#buyInfo > a')))
        info_button.click()
        
        # 용량 추출
        volume = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[2]/dd'))).text
        
        # 제품주요사양(스킨타입) 추출
        skin_type = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[3]/dd'))).text
        
        try:
        # 성분 추출
            ingredient = WebDriverWait(driver, 4).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="artcInfo"]/dl[8]/dd'))).text
        except:
            ingredient = None
            pass
        # 리뷰 버튼 클릭
        review_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#reviewInfo > a')))
        review_button.click()
        review_count = int(driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text.replace('(', '').replace(')', '').replace(',', ''))
        if review_count >=100:
            # 각 제품의 정보 출력
            indiv_product_reviews = extract_review(driver, name)
        
            # 한 제품에 대한 리스트 만들기
            make_list(name, brand, price, sale_price, picture, url, volume, skin_type, ingredient, indiv_product_reviews)
        else:
            pass
        driver.back()
        
    except NoSuchElementException:
        driver.back()



try:
    # 크롤링을 중지할 조건
    # True로 변경되면 중지
    stop = False
    # 페이지 1~8
    for page in range(1, 8):
        if stop:
            break
        for para in range(8,20):
            if stop:
                break
            for product_number in range(1, 5):
                if stop:
                    break
                url = f'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010013&fltDispCatNo=&prdSort=01&pageIdx={page}&rowsPerPage=48&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat100000100010013_Small&amplitudePageGubun=&t_page=&t_click=&midCategory=%EC%8A%A4%ED%82%A8%2F%ED%86%A0%EB%84%88&smallCategory=%EC%A0%84%EC%B2%B4&checkBrnds=&lastChkBrnd='
                
                driver.get(url)

                try:
                    extract_data(driver, para, product_number)
                except NoSuchElementException:
                    stop = True
                    break
    # 데이터프레임 생성 및 출력
    info_data = {'name': name_list,'brand': brand_list, 'price': price_list, 'sale_price': sale_price_list, 'picture':picture_list, 'url' : url_list, 'volume': volume_list, 'skin_type': skin_type_list, 'ingredient': ingredient_list, 'review' : review_list}
    df_info = pd.DataFrame(info_data)

finally:
    # 브라우저 종료
    driver.quit()



In [ ]:
df_info

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,[굿즈증정/1+1]닥터지 레드 블레미쉬 클리어 수딩 크림 70ml 1+1 기획,닥터지,"38,000","28,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,70ml+70ml+인사이드아웃 콜라보 시온컵,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...","[[여름용 크림으로 라이트한 걸 찾던 중 샀어요. 아침용으로 쓰기 좋아요], [좋아..."
1,라로슈포제 시카플라스트 밤 B5+ 100ml 기획 (+3ml 추가증정),라로슈포제,"39,000","31,590",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,100ml + 3ml,민감성 및 모든 피부에 사용 적합,정제수 하이드로제네이티드폴리아이소부텐 다이메티콘 글리세린 시어버터 판테놀 옥수수전분...,"[[[옵션] 시카플라스트밤 100ml 기획], [[옵션] [기획] 시카밤 100ml..."
2,[수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 기획 (60ml+30ml),에스네이처,"24,000","17,010",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,60ml(+30ml),모든 피부 타입,"정제수, 스쿠알란(150,000ppm), 글리세린, 1,2-헥산다이올, 베타인, 판...","[[무겁지 않고 수분감을 가득 채워줘서 여름에도 편하게 사용하기 좋아요], [유분이..."
3,[수딩진정]넘버즈인 1번 판토텐산 액티브 수딩크림 80ml+80ml 더블 기획,넘버즈인,"40,000","23,560",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,80ml*2,모든 피부 용,"정제수, 부틸렌글라이콜, 글리세린, 나이아신아마이드, 세틸에틸헥사노에이트, 다이카프...","[[리뷰가 좋길래 한번 사봤어요!\n제발 잘 맞았으면 좋겠어요ㅠ], [다른 제품 쓰..."
4,[화해1위/수분쿨링진정] 에스네이처 아쿠아 오아시스 수분 젤크림 90ml 기획 (+...,에스네이처,"34,000","20,610",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,"젤크림 90ml, 카밍 패드 2매(6ml)",모든 피부 타입,"젤크림 : 정제수, 글리세린, 부틸렌글라이콜, 1,2-헥산다이올, 블루아가베잎추출물...",[[쿨링감도 있고 피부에 흡수도 잘되고 자극없이\n사용할수 있어서 좋아요.\n생각보...
...,...,...,...,...,...,...,...,...,...,...
170,[단독기획] 나인위시스 비건 하이드라 앰플 수딩 크림 50ml 더블기획,나인위시스,"28,000","19,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,50ml+50ml,모든 피부 타입,"정제수, 부틸렌글라이콜, 글리세린, 미네랄워터, 1,2-헥산다이올, 다마스크장미꽃수...",[[운동하고나서 꼭 바르는 수분크림! 벌써 몇통이나 비웟는지 몰라요 바르고 나면 짱...
171,유리아쥬 배리어덤 시카 크림 40ml,유리아쥬,"24,000","19,200",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,40ml,민감 피부 [모든 피부 타입],"[배리어덤 시카 크림]온천수*,정제수,미네랄오일,소듐클로라이드,징크옥사이드,세틸피이...","[[이거로 제 료션은 정착합니다 ㅎㅎ 뭐하나 빠지는게 있다면 유분기에요 ㅠㅋㅋ], ..."
172,클라뷰 리얼 비건 저분자 콜라겐 탄력 크림 50ml,클라뷰,"31,000","18,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,50ml,모든 피부 타입,"정제수, 글리세린, 카프릴릭/카프릭트라이글리세라이드, 비닐다이메티콘, 세틸에틸헥사노...",[[건성보다는 복합성이 좋은거같아요 기능성이다보니 매일은 사용하지않고 보습이 짱짱한...
173,바이오더마 세비엄 센서티브 30ml,바이오더마,"24,000","18,000",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30ml,중복합 및 모든 피부,"정제수, 글리세린, 메틸메타크릴레이트크로스폴리머, 다이프로필렌글라이콜, 징크글루코네...","[[화장도 안 밀리고 여드름성 피부들이 바르기에 딱 좋은 것 같아요!], [남자친구..."


In [13]:
48*5

240

In [ ]:
df_info.to_csv('skin_toner_review_2.csv')

In [ ]:
df_all_review[:]

,review
0,[대용량에 화장솜까지 구성되어 있어서 구매했는데 순하고 막쓰기 좋아요!! 화장솜도 ...
1,[좋아요\n순하고 양이 많아서 한 번 사두면 팍팍 쓰기 좋습니다!]
2,[물같은 토너를 찾고있었는데 딱 좋습니다 이벤트로 200ml 더 주는게 있었는데 놓...
3,"[스킨팩하려고 샀는데 양도 많아서 잘 안줄어요ㅎㅎ\n무향이고 지성, 복합성에 적절한..."
4,"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
...,...
3567,[사용한지 벌써 4년...? 넘은 것 같네요 어쩌다 다른 브랜드 토너 썼다가도 다시...
3568,[요즘 너무 피부 여드름도 올라오고 울긋불긋해서\n진정에 좋은 토너 뭐가있을까보다가...
3569,[피부타입: 건조함\n\n구매동기 : 각질정돈과 트러블케어에 좋다고 해서 구매했습니...
3570,[그냥 제가 안맞는 것 같지만…\n뭐가 은근슬쩍 나는 느낌…\n유명해서 한번 사봤는...


In [ ]:
df_all_review.review.drop_duplicates()

0       [대용량에 화장솜까지 구성되어 있어서 구매했는데 순하고 막쓰기 좋아요!! 화장솜도 ...
1                   [좋아요\n순하고 양이 많아서 한 번 사두면 팍팍 쓰기 좋습니다!]
2       [물같은 토너를 찾고있었는데 딱 좋습니다 이벤트로 200ml 더 주는게 있었는데 놓...
3       [스킨팩하려고 샀는데 양도 많아서 잘 안줄어요ㅎㅎ\n무향이고 지성, 복합성에 적절한...
4       [※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ...
                              ...                        
3567    [사용한지 벌써 4년...? 넘은 것 같네요 어쩌다 다른 브랜드 토너 썼다가도 다시...
3568    [요즘 너무 피부 여드름도 올라오고 울긋불긋해서\n진정에 좋은 토너 뭐가있을까보다가...
3569    [피부타입: 건조함\n\n구매동기 : 각질정돈과 트러블케어에 좋다고 해서 구매했습니...
3570    [그냥 제가 안맞는 것 같지만…\n뭐가 은근슬쩍 나는 느낌…\n유명해서 한번 사봤는...
3571    [다른 토너 몇십개 써봤지만 챱토용으로 좋은 건 이게 가성비 적으로도 무난하고 참 ...
Name: review, Length: 3174, dtype: object